In [1]:
import argparse
import json
import os

import numpy as np
import pytorch_lightning as L
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader
from transformers import (
    ConditionalDetrConfig,
    ConditionalDetrForObjectDetection,
    ConditionalDetrForSegmentation,
    DeformableDetrConfig,
    DeformableDetrForObjectDetection,
    DetrConfig,
    DetrForObjectDetection,
    DetrForSegmentation,
    DetrImageProcessor,
)

import modules
import utils

/home/feity/anaconda3/envs/pytorch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = "/home/feity/cryoem/checkpoints/deformable_detr_pretrain"
with open(os.path.join(path, "config.json"), "r") as f:
    configs = json.load(f)

In [3]:
if configs["model"]["name"] == "conditional_detr":
    if configs["model"]["task"] == "segmentation":
        config = ConditionalDetrConfig(
            use_pretrained_backbone=False, **configs["model"]["args"]
        )
        seg_model = ConditionalDetrForSegmentation(config)
        if len(configs["model"]["pretrained"]) > 0:
            model = ConditionalDetrForObjectDetection.from_pretrained(
                configs["model"]["pretrained"],
                ignore_mismatched_sizes=True,
                **configs["model"]["args"]
            )
            seg_model.conditional_detr.load_state_dict(model.state_dict())
        model = seg_model
    elif configs["model"]["task"] == "detection":
        if len(configs["model"]["pretrained"]) > 0:
            model = ConditionalDetrForObjectDetection.from_pretrained(
                configs["model"]["pretrained"],
                ignore_mismatched_sizes=True,
                **configs["model"]["args"]
            )
        else:
            config = ConditionalDetrConfig(
                use_pretrained_backbone=False, **configs["model"]["args"]
            )
            model = ConditionalDetrForObjectDetection(config)

elif configs["model"]["name"] == "deformable_detr":
    if len(configs["model"]["pretrained"]) > 0:
        model = DeformableDetrForObjectDetection.from_pretrained(
            configs["model"]["pretrained"],
            ignore_mismatched_sizes=True,
            **configs["model"]["args"]
        )
    else:
        config = DeformableDetrConfig(
            use_pretrained_backbone=False, **configs["model"]["args"]
        )
        model = DeformableDetrForObjectDetection(config)

elif configs["model"]["name"] == "detr":
    if configs["model"]["task"] == "segmentation":
        if len(configs["model"]["pretrained"]) > 0:
            model = DetrForSegmentation.from_pretrained(
                configs["model"]["pretrained"],
                ignore_mismatched_sizes=True,
                **configs["model"]["args"]
            )
        else:
            config = DetrConfig(
                use_pretrained_backbone=False, **configs["model"]["args"]
            )
            model = DetrForSegmentation(config)
    elif configs["model"]["task"] == "detection":
        if len(configs["model"]["pretrained"]) > 0:
            model = DetrForObjectDetection.from_pretrained(
                configs["model"]["pretrained"],
                ignore_mismatched_sizes=True,
                **configs["model"]["args"]
            )
        else:
            config = DetrConfig(
                use_pretrained_backbone=False, **configs["model"]["args"]
            )
            model = DetrForObjectDetection(config)
else:
    raise NotImplementedError

Some weights of the model checkpoint at SenseTime/deformable-detr were not used when initializing DeformableDetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DeformableDetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DeformableDetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DeformableDetrForObjectDetection wer

In [5]:
model = modules.Detr(
        lr=configs["training"]["lr"],
        lr_backbone=configs["training"]["lr_backbone"],
        weight_decay=configs["training"]["weight_decay"],
        model=model,
    )

In [7]:
ckpt = torch.load(os.path.join(path, "last.ckpt"))
model.load_state_dict(ckpt["state_dict"])

/tmp/ipykernel_278559/624496722.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(os.path.join(path, "last.ckpt"))


<All keys matched successfully>

In [8]:
torch.save(model.model.state_dict(), os.path.join(path, "model.pth"))

In [10]:
import argparse
import json
import os

import numpy as np
import pytorch_lightning as L
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader
from transformers import (
    ConditionalDetrConfig,
    ConditionalDetrForObjectDetection,
    ConditionalDetrForSegmentation,
    DeformableDetrConfig,
    DeformableDetrForObjectDetection,
    DetrConfig,
    DetrForObjectDetection,
    DetrForSegmentation,
    DetrImageProcessor,
)
import modules
import utils

In [11]:
path = "/home/feity/cryoem/checkpoints/deformable_train"
with open(os.path.join(path, "config.json"), "r") as f:
    configs = json.load(f)

In [12]:
if configs["data"]["transform"] == "default":
    t = utils.getDefaultTransform()
else:
    t = utils.getConstantTransform()

In [13]:
train_sets = {}
for i in configs["data"]["filter_class"]:
    train_sets[i] = modules.CocoDetection(
        configs["data"]["image_path"],
        configs["data"]["annotation_path_train"],
        is_npy=configs["data"]["is_npy"],
        transform=t,
        require_mask=configs["data"]["require_mask"][i],
        filter_class=configs["data"]["filter_class"][i],
        single_class=configs["data"]["single_class"][i],
        mark=i,
    )
val_sets = {}
for i in configs["data"]["filter_class"]:
    val_sets[i] = modules.CocoDetection(
        configs["data"]["image_path"],
        configs["data"]["annotation_path_val"],
        is_npy=configs["data"]["is_npy"],
        transform=utils.getConstantTransform(),
        require_mask=configs["data"]["require_mask"][i],
        filter_class=configs["data"]["filter_class"][i],
        single_class=configs["data"]["single_class"][i],
        mark=i,
    )

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [14]:
ds = modules.EMDataModule(
        train_sets,
        val_sets,
        configs["training"]["train_batch_size"],
        configs["training"]["val_batch_size"],
    )


In [15]:
d = ds.train_dataloader()

In [19]:
for i, j in enumerate(d[1]):
    print(j)
    break


{'pixel_values': tensor([[[[0.5881, 0.6217, 0.6295,  ..., 0.0000, 0.0000, 0.0000],
          [0.6093, 0.6250, 0.6303,  ..., 0.0000, 0.0000, 0.0000],
          [0.6388, 0.6335, 0.6370,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.6120, 0.6620, 0.6210,  ..., 0.0000, 0.0000, 0.0000],
          [0.6380, 0.6364, 0.6223,  ..., 0.0000, 0.0000, 0.0000],
          [0.6515, 0.6273, 0.6154,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.5732, 0.6074, 0.6211,  ..., 0.0000, 0.0000, 0.0000],
          [0.5706, 0.6052, 0.6266,  ..., 0.0000, 0.0000, 0.0000],
          [0.6095, 0.6277

In [1]:
import pickle 
with open("./temp.pkl", "rb") as f:
    a = pickle.load(f)

In [3]:
a[0]

{'pixel_values': tensor([[[[0.6563, 0.6545, 0.6513,  ..., 0.0000, 0.0000, 0.0000],
           [0.6610, 0.6569, 0.6514,  ..., 0.0000, 0.0000, 0.0000],
           [0.6649, 0.6623, 0.6523,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
          [[0.6620, 0.6623, 0.6564,  ..., 0.0000, 0.0000, 0.0000],
           [0.6595, 0.6591, 0.6646,  ..., 0.0000, 0.0000, 0.0000],
           [0.6524, 0.6597, 0.6588,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
          [[0.6596, 0.6614, 0.6615,  ..., 0.0000, 0.0000, 0.0000],
           [0.6599, 0.6620, 0.6628,  ..., 0.0000, 0.0000, 0.0000],
        